In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

cfg

In [3]:
import torch
import sys
sys.path.append('../..')
from functions.superpoint import SPG
from omegaconf import OmegaConf

In [4]:
cfg = OmegaConf.load("spg.yaml")

In [5]:
cfg.Knn.k

45

transforms

In [6]:
data_dict = torch.load("scene0000_00.pth")
data_dict.keys()

dict_keys(['coord', 'color', 'scene_id', 'normal', 'semantic_gt20', 'semantic_gt200', 'instance_gt'])

In [7]:
spg = SPG(cfg)
spg

In [8]:
data = dict(pos=torch.from_numpy(data_dict["coord"]), rgb=torch.from_numpy(data_dict["color"]))
data.keys()

dict_keys(['pos', 'rgb'])

In [9]:
data = spg.Knn(data)
data.keys(), data["neighbor_index"].shape, data["neighbor_distance"].shape

(dict_keys(['pos', 'rgb', 'batch', 'neighbor_index', 'neighbor_distance']),
 torch.Size([81369, 45]),
 torch.Size([81369, 45]))

In [10]:
data = spg.GroundElevation(data)
data.keys(), data['elevation'].shape

(dict_keys(['pos', 'rgb', 'batch', 'neighbor_index', 'neighbor_distance', 'elevation']),
 torch.Size([81369, 1]))

In [11]:
data = spg.PointFeatures(data)
data.keys()

dict_keys(['pos', 'rgb', 'batch', 'neighbor_index', 'neighbor_distance', 'elevation', 'density', 'linearity', 'planarity', 'scattering', 'verticality', 'curvature', 'length', 'surface', 'volume', 'normal'])

In [12]:
data = spg.AdjacencyGraph(data)
data.keys()

dict_keys(['pos', 'rgb', 'batch', 'neighbor_index', 'neighbor_distance', 'elevation', 'density', 'linearity', 'planarity', 'scattering', 'verticality', 'curvature', 'length', 'surface', 'volume', 'normal', 'edge_index', 'edge_attr'])

In [13]:
data = spg.graph_connect_isolated(data)
data = spg.AddKeysTo(data, spg.cfg.partition_hf, "x")
data = spg.graph_to_trimmed(data)

In [14]:
len(cfg.CutPursuitPartition.spatial_weight)

3

In [15]:
data["x"].shape

torch.Size([81369, 8])

In [16]:
data_list = spg.CutPursuitPartition(data)

In [17]:
torch.unique(data_list[0]["super_index"]).shape, torch.unique(data_list[1]["super_index"]).shape, torch.unique(data_list[2]["super_index"]).shape, data_list[3]["pos"].shape[0]

(torch.Size([3138]), torch.Size([3138]), torch.Size([3137]), 3137)

In [18]:
data_list[0]["super_index"], data_list[0]["super_index"].shape

(tensor([1923,  167,  529,  ..., 2011, 2553, 2553]), torch.Size([81369]))

给出三个 level 的 superpoint cluster

TODO: 其实只要level 0就够下游应用了，需要精简一下节省时间，比如 compute_Graph、compute_List 什么的没必要计算，详见 `cp_d0_dist_cpy.cpp`